# Day 1 to 4:
- 1. Load Dataset & Quick Scan/Exploration
- 2. Data Clean
- 3. Data Analysis
- 4. Feature Engieering & Outlier Detection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
pd.set_option('display.max_columns', None)

## Data Load & Quick Scan

In [ ]:
import kagglehub
path = kagglehub.dataset_download('blastchar/telco-customer-churn')

In [ ]:
filename = os.listdir(path)[0]
fp = os.path.join(path, filename)

In [ ]:
df = pd.read_csv(fp)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

## Data Clean

In [ ]:
missing = df.isna().sum().to_frame('missing_count')
missing['missing_pct'] = (missing['missing_count'] / len(df)) * 100
print("Missing rows:\n", missing)

In [ ]:
duplicates = df.duplicated().sum()
print("Duplicate rows:\n", duplicates)

In [ ]:
# Fix TotalCharges column
# print(df['TotalCharges'].dtype)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# print(df['TotalCharges'].dtype)

In [ ]:
df['TotalCharges'].isna().sum() # 11

In [ ]:
df = df[df['TotalCharges'].notna()].copy()

In [ ]:
df['TotalCharges'].isna().sum() # 0

In [ ]:
unique_vals = df.nunique()
unique_vals

In [ ]:
cat_threshold = 4
low_cardinality_cols = unique_vals[unique_vals <= cat_threshold].index
print("Low Cardinality Columns:", len(low_cardinality_cols), "\n", low_cardinality_cols)

In [ ]:
df[low_cardinality_cols] = df[low_cardinality_cols].astype('category')
df.dtypes

In [ ]:
internet_related_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'
]
for col in internet_related_cols:
  df[col] = df[col].replace({'No internet service': 'No'})

df['MultipleLines'] = df['MultipleLines'].replace({'No phone service': 'No'})

In [ ]:
for col in internet_related_cols + ['MultipleLines']:
  print(col, df[col].nunique())

In [ ]:
# final validation
df.isna().sum()

In [ ]:
# export cleaned dataset
os.makedirs("data", exist_ok=True)
df.to_csv("data/cleaned_dataset_v1.csv", index=False)

## Data Analysis
- Basic Churn Summary
- Churn by Demographics (The Person / User attributes)
- Churn by Subscription / Services (The Product / Usage attributes)
- Churn by Account (The Relationship / Administrative attributes)
- Tenure Segmentation (cohort-style buckets/bins)

In [ ]:
# Basic Churn Summary

churn_summary = df['Churn'].value_counts().to_frame('count')
# churn_summary
churn_summary['percent'] = round((churn_summary['count'] / len(df)) * 100, 2)
churn_summary

In [ ]:
# Churn by Demographics

demographic_cols = ['gender', 'SeniorCitizen', 'Dependents']

demographic_churn = {}

for col in demographic_cols:
  table = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
  demographic_churn[col] = table
  print(f"*** {col} vs Churn (%) ***\n{table}\n")

Observations:
- gender -> no effect
- senior citizens -> very high churn (2x)
- dependents:
  - with -> low churn
  - without -> high churn (2x)

In [ ]:
# Churn by Subscription / Services

service_cols = [
    'PhoneService', 'MultipleLines', 'InternetService',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

service_churn = {}

for col in service_cols:
  table = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
  service_churn[col] = table
  print(f"*** {col} vs Churn (%) ***\n{table}\n")

Observations:
- Fiber-optic customers -> very high churn
- Streaming -> high churn
- Other Services:
  - with -> low churn
  - without -> high churn

In [ ]:
# Churn by Account

account_cols = ['Contract', 'PaymentMethod']

account_churn = {}

for col in account_cols:
  table = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
  account_churn[col] = table
  print(f"*** {col} vs Churn (%) ***\n{table}\n")

Observations:
- Month-to-month -> very high churn
- Electronic check -> very high churn

In [ ]:
# Tenure Segmentation (cohort-style buckets/bins)
bins = [x for x in range(0, 73, 12)]

labels = ['0-12', '13-24', '25-36', '37-48', '49-60', '61-72']

df['tenure_group'] = pd.cut(df['tenure'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
tenure_churn = pd.crosstab(df['tenure_group'], df['Churn'], normalize='index') * 100
tenure_churn

Observations:
- new customers (0-12 months) -> very high churn
- old/long-term customers (61-72 months) -> lowest churn

In [ ]:
# Revenue Metrics
# ARPU by Churn: average monthly revenue (churner vs non-churner)
arpu_summary = round(df.groupby('Churn')['MonthlyCharges'].mean(), 2)
arpu_summary

In [ ]:
# Lifetime Value by Churn

df['LTV'] = df['MonthlyCharges'] * df['tenure']

ltv_summary = round(df.groupby('Churn')['LTV'].mean(), 2)
ltv_summary

In [ ]:
# ARPU by contract type

arpu_by_contract = round(df.groupby('Contract')['MonthlyCharges'].mean(), 2)
arpu_by_contract

Observations:
- Customers who pay more churn more, but also generate less lifetime value -> not loyal, high ARPU, low LTV
- Customers who stay loyal, pay less each month (~5.5 per user per month compared to monthly customers), but generate high lifetime value -> loyal, low ARPU, high LTV
- Month-to-month customers pay more, but also churn most

## Feature Engineering & Outlier Detection

### Step 1 — Outlier Detection (MonthlyCharges, TotalCharges, LTV)

In [ ]:
num_cols = ['MonthlyCharges', 'TotalCharges', 'LTV']

In [ ]:
for col in num_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot - {col}")
    plt.show()

    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr

    print(f"{col}: Lower={lower}, Upper:{upper}")
    print(f"Outliers Count: {df[(df[col] < lower) | (df[col] > upper)].shape[0]}")

### Step 2 — Tenure Buckets (categorical feature)

In [ ]:
# df['tenure_group'].unique()
df['tenure_group'] = df['tenure_group'].astype('category')
df['tenure_group'] = df['tenure_group'].cat.reorder_categories(['0-12','13-24','25-36','37-48','49-60','61-72'], ordered=True)

df['tenure_group'].value_counts()

Notes: `reorder_categories` is Necessary to establish an **Ordinal Relationship** among the tenure groups.

- Tenure buckets are ordinal (0–12 < 13–24 < ...).
- pd.cut may lose ordering when converted to category.
- `reorder_categories` ensures correct ordinal order for ML models.

### Step 3 — ARPU Tiers (Low / Medium / High revenue)
- create revenue tiers using quantiles
- 3-level segmentation, useful for EDA & ML

In [ ]:
# Quantile-based discretization function

df['ARPU_tier'] = pd.qcut(df['MonthlyCharges'], q=3, labels=['Low', 'Medium', 'High'])

df['ARPU_tier'] = df['ARPU_tier'].astype('category')

df['ARPU_tier'].value_counts()

### Step 4 — Risk Flags (high risk payment method, fiber, no security, etc.)
- convert the strongest churn predictors into direct risk flags.

In [ ]:
# High-risk payment (electronic check users)

df['is_electronic_check'] = (df['PaymentMethod'] == 'Electronic check').astype(int)

df['is_electronic_check'].value_counts()

In [ ]:
# High-risk contract type (month-to-month)

df['is_monthly_contract'] = (df['Contract'] == 'Month-to-month').astype('int')

df['is_monthly_contract'].value_counts()

In [ ]:
# High-risk internet service (Fiber optic)

df['is_fiber'] = (df['InternetService'] == 'Fiber optic').astype('int')

df['is_fiber'].value_counts()

In [ ]:
# Missing protection services

df['no_tech_support'] = (df['TechSupport'] == 'No').astype('int')
df['no_online_security'] = (df['OnlineSecurity'] == 'No').astype('int')
df['no_device_protection'] = (df['DeviceProtection'] == 'No').astype('int')

In [ ]:
# Senior citizen risk flag (strong demographic predictor)

df['is_senior'] = (df['SeniorCitizen'] == 1).astype('int')
df['is_senior'].value_counts()

### Step 5 — Ratio Features (Value-Perception Metrics)

In [ ]:
# Cost per month of tenure (stability indicator) — historical average

df['avg_cost_per_month'] = df['TotalCharges'] / df['tenure'].replace(0, 1)

In [ ]:
# Protection-to-cost ratio (expected value indicator)

df['security_to_cost_ratio'] = df['no_online_security'] / df['MonthlyCharges']

Notes: `security_to_cost_ratio` highlights churn risk:

- higher ratio = low-cost, unprotected customers (highest churn tendency)
- low ratio = high-cost but unprotected (still risky)
- 0 ratio = customers with security (low risk)

Helps model detect vulnerable, poor-value customers.


In [ ]:
# Contract value proxy (monthly charges × contract length assumption)

contract_map = {
    'Month-to-month': 1,
    'One year': 12,
    'Two year': 3
}

df['contract_length'] = df['Contract'].map(contract_map)
# CONVERT TO INT, else below line throws a TypeError: Object with dtype category cannot perform the numpy op multiply
df['contract_length'] = df['contract_length'].astype('int')

df['contract_value_proxy'] = df['MonthlyCharges'] * df['contract_length']

### Step 6 — Export cleaned_featured_dataset_v2.csv

In [ ]:
# Final Validation

print(f"{df.shape}\n{'*** ' * 10}\n{df.head(3)}\n{'*** ' * 10}\n{list(df.columns)}")

In [ ]:
os.makedirs("data", exist_ok=True)
df.to_csv("data/cleaned_featured_dataset_v2.csv", index=False)